In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import random
import statistics as stat

In [4]:
import numpy as np
import math
import copy

In [5]:
# first we want to approach the EMS as this model will rely on the behavior of the reduced order data centers. 
# Their energy requirements will be determined by data from a higher order model and then tuned by a ANN.
# Since I'll only likely have time for one digital twin this fall, I'll create a random data set that will serve as the 
# information the ANN uses to dictate low level data center models. 

# What's needed:
# -Neural net to train devices to the behavior of more complex models


In [57]:
# N - number of devices connected to the supplier
# locations - Nx2 list -  positions of each device, will be renamed  
#             after check to see if empty. If empty -> rand gen
# bounds - 2x2 list - x and y bounds of where devices can be located [xlow, xhigh; ylow, yhigh]

def emsSIM(N, locations, bounds,w1,w2,w3):
    
    # define a bunch of the variables
    # megawatts
    MJ = 1000000
    connectivitylevel = .5
    targetBase = 1e6
    ampTarget = 1e4

    cState0 = 0
    sState0 = 0
    
    #loss per unit length
    lossPUL = .05
    
    nCustomers  = N
    
    # why sizes?
    csize = 1.5
    ssize = 3
    
    # variation of position amplitudes
    sAmpx = 0
    sAmpy = 0
    sAmpz = 0
    
    # base values
    
    cgenbase = 1e4
    cConsumedbase = 1e6
    cIcJinterFluxBase = 1e5
    
    # define matrices
    targ = np.ones(nCustomers)
    supply = np.zeros(nCustomers)
    resupply = np.zeros(nCustomers)
    
    # target values for local units
    targ = targetBase + (ampTarget*((np.random.rand(N)-0.5)*2))
        
    # i to j connectivity, zohdi calls it cicjon, whether or not two devices are connected
    ijcon = np.zeros([nCustomers,nCustomers])
    
    for i in range(nCustomers):
        for j in range(nCustomers):
            if i != j:
                rand = np.random.rand(1)
                
                if rand <= connectivitylevel:
                    ijcon[i][j] = 1
                    ijcon[j][i] = 1
                    
    # initial customer conditions
    cstateT = cState0*np.ones(nCustomers)
    cstateTpDT = cState0*np.ones(nCustomers)
    
    #initial storage unit system conditions
    sstateT = sState0
    sstateTpDT = sState0
     
    # function for overall sim will go here, lets define supporting 
    # functions first
    
    # the processes 
    timelimit = 100
    time = 0
    deltaT = 1
    lasttime = 0
    movieframes = 100
    totaldemand = 0
    totallosses = 0
    totalsupplylosses = 0
    totalcost = 0
    
    # Create positions of devices and supplier
    # position of EMS will always be origin, for now (randomly set to z = -5, need to ask Zohdi)
    sup_pos = np.array([0,-5,0],'float')
    
    if not locations:  # check that locations is an empty list
        
        # make positions an array of zeros to be filled in size: Nx3
        positions = np.array([[0] * 3] * N,'float')
        for i in range(N):    # loop through 
            positions[i][0] = bounds[0][0] + (bounds[0][1] - bounds[0][0])*np.random.rand()    # x min + x range * 0<=rand<=1
            positions[i][2] = bounds[1][0] + (bounds[1][1] - bounds[1][0])*np.random.rand()    # y min + y range * 0<=rand<=1
            positions[i][1] = 0
            
    else: # if locations has an input, then user input will decide positions of data centers
        positions = locations

    # THIS IS WHERE THE PHYSICS MODELING BEGINS

    while time <= timelimit:
        # pre-simulation energies for each device
        initialE = np.random.rand(1,N)
        # generate target energy each device 
        targetE = np.random.rand(1,N)
        # generate uniform target energy in each device
        # use targ

        # generate energy consumed by each device
        Econ = np.random.rand(1,N)
        # generate energy generated by each device
        Egen = np.random.rand(1,N)

        # flux between devices -- THIS IS WHAT WE MANIPULATE IN THE GA
        interflux = interFlux(nCustomers,cIcJinterFluxBase)
        # customer state calculation
        cgen,cconsume = customerState(nCustomers,cgenbase,cConsumedbase)
        losses1 = 0
        losses2 = 0
        penalty = 0 # peanlty for devices with negative energy
        # loop through all customer-customer pairs to 
        for i in range(nCustomers):
            sum1 = 0
            for j in range(nCustomers):
                if i <= j:
                    # distance does not include y values since all devices are assume to be on a flat plane, this can be changed later
                    distance= np.sqrt( (positions[i][0] - positions[j][0])**2 + (positions[i][2] - positions[j][2])**2) 
                if ijcon[i][j]*interflux[i][j] > 0:
                    alphaij = 1
                elif ijcon[i][j]*interflux[i][j] <= 0:
                    alphaij = math.exp(-lossPUL*distance) # power received by device with less power from device with more
                
                sum1 = ijcon[i][j]*interflux[i][j]*alphaij + sum1  # sum of power transferred to or from device i
                losses1 = abs(ijcon[i][j]*interflux[i][j])*(1-alphaij)*deltaT+losses1  # losses from device to device
            # end j for loop
            
            # local supply needed calculation
            # gamma determines whether device i needs from or will supply power to the supplier 
            gamma = (targ[i]-cstateT[i])/deltaT - (cgen[0][i]-cconsume[0][i]-sum1)
            distance = np.sqrt( (positions[i][0] - sup_pos[0])**2 + (positions[i][1] - sup_pos[1])**2 + (positions[i][2] - sup_pos[2])**2)
            
            # Needs energy from supplier, overcompensate for control volume Supply
            if gamma > 0:
                supply[i] = gamma*math.exp(lossPUL*distance)
                resupply[i] = 0
                losses2 = abs(supply[i])*(1-math.exp(-lossPUL*distance))*deltaT+losses2
            
            # send supplies, include losses
            if gamma <= 0:
                resupply[i] = gamma*math.exp(-lossPUL*distance)
                supply[i] = 0
                losses2 = abs(gamma)*(1-math.exp(-lossPUL*distance))*deltaT + losses2
            
            # Local state calculation
            cstateTpDT[i] = cstateT[i] + deltaT*(cgen[0][i]-cconsume[0][i]-sum1+gamma)
            
            # update customer state
            cstateT[i] = cstateTpDT[i]

            if cstateT[i] < 1e4:
                    penalty = penalty + (1e4 - cstateT[i])**2
        
        # end i for loop
        # PHYSICS MODELING ENDS
        
        # COST IS CALCULATED
        # update storage unit vontrol volume S
        sum1 = 0 
        for i in range(nCustomers):
            sum1 = (supply[i] + resupply[i])+sum1
        # end for loop
        
        # storage unit state
        sstateTpDT = sstateT - deltaT*sum1
        sstateT = sstateTpDT
        
        # quantities for cost function
        
        totaldemand = -sum1*deltaT + totaldemand
        totallosses = totallosses + (losses1+losses2)
        totalsupplylosses  = totalsupplylosses + losses2
        
        # skip graphics portion
        
        # totalcost builds on each timestep
        totalcost = totalcost + w1*totallosses/MJ + w2*totalsupplylosses/MJ + w3*totaldemand/MJ + penalty
        
        # update time step
        time = deltaT + time
        
    return totalcost
    # end while loop for time
    
    # what is override?

In [7]:
B = np.array([[5] * 3] ,'float')
T = 5*np.ones(3)
T==B

array([[ True,  True,  True]])

In [52]:
# function to generate positions of devices, can also be a user input
#bounds is a 2x2

def POSITIONS(N,locations,bounds):
    
    # position of EMS will always be origin, for now (set to y = -5, need to ask Zohdi)
    sup_pos = np.array([0,-5,0],'float')
    
    if not locations:  # check that locations is an empty list
        
        # make positions an array of zeros to be filled in size: Nx3
        positions = np.array([[0] * 3] * N,'float')
        for i in range(N):    # loop through 
            positions[i][0] = bounds[0][0] + (bounds[0][1] - bounds[0][0])*np.random.rand()    # x min + x range * 0<=rand<=1
            positions[i][2] = bounds[1][0] + (bounds[1][1] - bounds[1][0])*np.random.rand()    # y min + y range * 0<=rand<=1
            positions[i][1] = 0
            
    else: # if locations has an input, then user input will decide positions of data centers
        positions = locations
        
        # need to add check that no data centers have coordinates too close, ref summer project
    return positions,sup_pos
        

In [53]:


# write function to quantify flux between devices

def interFlux(nCustomers,cIcJinterFluxBase):
    interflux = np.zeros([nCustomers,nCustomers])
    for i in range(nCustomers):
        for j in range(i+1,nCustomers):
            interflux[i][j] = ((random.random()-.5)/.5) * cIcJinterFluxBase  # flux i to j
            interflux[j][i] = -interflux[i][j]          # flux j to i
    return interflux
            
# customer state calculations

def customerState(nCustomers,cgenbase,cConsumedbase):
    cgen = np.zeros([1,nCustomers])
    cconsume = np.zeros([1,nCustomers])
    for i in range(nCustomers):
        pop = (random.random()-0.5)/0.5
        cgen[0][i] = cgenbase*pop
        pop = (random.random()-0.5)/0.5
        cconsume[0][i] = cConsumedbase*pop
    return cgen, cconsume
        


In [61]:
emsSIM(20, [], [[-10, 10],[-10, 10]], .3, .4, .3)

11140.380879730712

In [11]:
a = np.zeros([10,2])
for x in range(3,10):
    a[x][1] = 5
a

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 5.],
       [0., 5.],
       [0., 5.],
       [0., 5.],
       [0., 5.],
       [0., 5.],
       [0., 5.]])

In [12]:
a=np.zeros([3,3])
for x in range(3):
    for y in range(x+1,3):
        a[x][y] = x+y
        a[y][x] = -a[x][y]
        
a

array([[ 0.,  1.,  2.],
       [-1.,  0.,  3.],
       [-2., -3.,  0.]])

In [13]:
### GENETIC ALGORITHM

# goal is to minimize line losses, supply losses, and overall demand
# lambda will be a vector of matrices

def emsGA(G,S,P,C,minTol,N,w1,w2,w3,deltaT,initialE,targetE,Econ,Egen,
          positions,ijcon,targ,cIcJinterFluxBase):
    
    MJ = 1000000
    totalcostGA = 0  # initialize totalcost as 0
    connectivitylevel = .5
    targetBase = 1
    ampTarget = .01
    nCustomers = N
    totaldemand = 0
    totallosses = 0
    totalsupplylosses = 0
    cgenbase = 100
    cConsumedbase = 100
    cIcJinterFluxBase = 100
    #loss per unit length
    lossPUL = .05
    sup_pos = np.array([0,-5,0],'float')
    supply = np.zeros(nCustomers)
    resupply = np.zeros(nCustomers)
    # initial customer conditions
    cstateT = np.zeros(nCustomers)
    cstateTpDT = np.zeros(nCustomers)
    
    #initial storage unit system conditions
    sstateT = 0
    sstateTpDT = 0
    
    numlam = math.floor((N-1)*(N)/2 + N )
    # number of variables in lambda is number of energy sent between each device pair 
    # plus energy sent from supplier to each device
    # eg 5 devices, 10 pairs --> AB,AC,AD,AE,BC,BD,BE,CD,CE,DE
    
    
    # form lambda
    lam = np.zeros([S,nCustomers,nCustomers])
    for k in range(S):
        for i in range(nCustomers):
            for j in range(i+1,nCustomers):
                lam[k][i][j] = ((random.random()-.5)/.5) * cIcJinterFluxBase  # flux i to j
                lam[k][j][i] = -lam[k][i][j]          # flux j to i
    

    # lambda has been formed
    # lets get genetic
    
    start = 0
    Pi = np.zeros([S,1])
    Pi_i = np.zeros([S,G]) # Pi index
    Min = np.zeros([G,1])
    pAve = np.zeros([G,1])
    totAve = np.zeros([G,1])
    

    tol = 1
    Gen = 0


    while tol > minTol and Gen < G:
        totaldemand = 0
        totallosses = 0
        totalsupplylosses = 0
        penalty=0
        supply = np.zeros(nCustomers)
        resupply = np.zeros(nCustomers)
        # initial customer conditions
        cstateT = np.zeros(nCustomers)
        cstateTpDT = np.zeros(nCustomers)

        #initial storage unit system conditions
        sstateT = 0
        sstateTpDT = 0
        
        for k in range(start,S):

            cgen,cconsume = customerState(nCustomers,cgenbase,cConsumedbase)
            losses1 = 0
            losses2 = 0

            # loop through all customer-customer pairs to 
            for i in range(nCustomers):
                sum1 = 0
                for j in range(nCustomers):
                    if i <= j:
                        # distance does not include y values since all devices are assume to be on a flat plane, this can be changed later
                        distance= np.sqrt( (positions[i][0] - positions[j][0])**2 + (positions[i][2] - positions[j][2])**2) 
                    if ijcon[i][j]*lam[k][i][j] > 0:
                        alphaij = 1
                    elif ijcon[i][j]*lam[k][i][j] <= 0:
                        alphaij = math.exp(-lossPUL*distance) # power received by device with less power from device with more

                    sum1 = ijcon[i][j]*lam[k][i][j]*alphaij + sum1  # sum of power transferred to or from device i
                    losses1 = abs(ijcon[i][j]*lam[k][i][j])*(1-alphaij)*deltaT+losses1  # losses from device to device
                # end j for loop

                # local supply needed calculation
                # gamma determines whether device i needs from or will supply power to the supplier 
                gamma = (targ[i]-cstateT[i])/deltaT - (cgen[0][i]-cconsume[0][i]-sum1)
                distance = np.sqrt( (positions[i][0] - sup_pos[0])**2 + (positions[i][1] - sup_pos[1])**2 + (positions[i][2] - sup_pos[2])**2)

                # Needs energy from supplier, overcompensate for control volume Supply
                if gamma > 0:
                    supply[i] = gamma*math.exp(lossPUL*distance)
                    resupply[i] = 0
                    losses2 = abs(supply[i])*(1-math.exp(-lossPUL*distance))*deltaT+losses2

                # send supplies, include losses
                if gamma <= 0:
                    resupply[i] = gamma*math.exp(-lossPUL*distance)
                    supply[i] = 0
                    losses2 = abs(gamma)*(1-math.exp(-lossPUL*distance))*deltaT + losses2

                # Local state calculation
                cstateTpDT[i] = cstateT[i] + deltaT*(cgen[0][i]-cconsume[0][i]-sum1+gamma)

                # update customer state
                cstateT[i] = cstateTpDT[i]
                if cstateT[i] < 1e4:
                    penalty = penalty + (1e4 - cstateT[i])**2

            # end i for loop
            
            # PHYSICS MODELING ENDS

            # COST CALCULATION
            # update storage unit vontrol volume S
            sum1 = 0 
            for i in range(nCustomers):
                sum1 = (supply[i] + resupply[i])+sum1
            # end i for loop

            # storage unit state
            sstateTpDT = sstateT - deltaT*sum1
            sstateT = sstateTpDT

            # quantities for cost function

            totaldemand = -sum1*deltaT + totaldemand
            totallosses = totallosses + (losses1+losses2)
            totalsupplylosses = totalsupplylosses+losses2

            # skip graphics portion

            totalcostGA =  w1*totallosses/MJ + w2*totalsupplylosses/MJ + w3*totaldemand/MJ + penalty
            Pi[k][0] = totalcostGA

        ind = (np.argsort(Pi,axis=0)).astype(int) # index of how to sort the Pi values from least to greatest

#         Pi = np.sort(Pi)  # sort Pi values
#         print(Pi)
        # create a copy of Pi andlam so we call on unmodified array when sorting strings
        PiNew = copy.copy(Pi)
        lamNew = copy.copy(lam)
        # order lam by best performing strings
        for kk in range(len(ind)):
            Pi[kk] = PiNew[ind[kk]]
            lam[kk,:,:] = lamNew[ind[kk],:,:]
        # end kk for loop
        
#         print(Pi)
        phi = np.random.rand(math.floor(C),nCustomers,nCustomers)

        parents = lam[0:P,:,:]  # save top P performing strings

        # np.arange(starting index, end {start + step must be less than end}, step)

        children = []
        for j in range(0,C,2):
            children.append(phi[j,:,:]*lam[j,:,:] + (1- phi[j,:,:])*lam[j+1,:,:])
            children.append(phi[j+1,:,:]*lam[j+1,:,:] + (1- phi[j+1,:,:])*lam[j,:,:])
        # end j for loop

        # generate mutation strings
        rando = np.zeros([S-P-C,nCustomers,nCustomers])
        for k in range(S-P-C):
            for i in range(nCustomers):
                for j in range(i+1,nCustomers):
                    rando[k][i][j] = ((random.random()-.5)/.5) * cIcJinterFluxBase  # flux i to j
                    rando[k][j][i] = -rando[k][i][j]          # flux j to i 

        
        lam[P:(P+C),:,:] = children
        lam[(P+C):S,:,:] = rando

        Pi_i[:,Gen] = Pi[:,0]
        Min[Gen][0] = Pi[0]
        pAve[Gen][0] = stat.mean(Pi[0:P,0])
        totAve[Gen][0] = stat.mean(Pi[:,0])
        Gen += 1
        start = P # this avoids recalculating the parameters for parents, whose cost is already known

        tol = Pi[0,0]
        
    # end while loop
    return Pi,Pi_i,Min,pAve,totAve, sstateT
# end function
        
        
        
        
            
                
        

In [14]:
def emsSIMwGA(N, locations, bounds,w1,w2,w3):
    
    # define a bunch of the variables
    # megawatts
    MJ = 1000000
    connectivitylevel = .5
    targetBase = 1
    ampTarget = .01

    cState0 = 0
    sState0 = 0
    
    #loss per unit length
    lossPUL = .05
    
    nCustomers  = N
    
    # why sizes?
    csize = 1.5
    ssize = 3
    
    # variation of position amplitudes
    sAmpx = 0
    sAmpy = 0
    sAmpz = 0
    
    # define variables for GA
    
    G = 10
    S = 12
    P = 4
    C = 4
    minTol = .0001
    
    # base values
    
    cgenbase = 100
    cConsumedbase = 100
    cIcJinterFluxBase = 100
    
    # define matrices
    targ = np.ones(nCustomers)
    supply = np.zeros(nCustomers)
    resupply = np.zeros(nCustomers)
    
    # target values for local units
    for i in range(nCustomers):
        pop = (np.random.rand(1) - .5) * 2
        targ[i] = targetBase*(1+ampTarget*pop)   # .99 <= targ <= 1.01 for all devices
        
    # i to j connectivity, zohdi calls it cicjon, whether or not two devices are connected
    ijcon = np.zeros([nCustomers,nCustomers])
    
    for i in range(nCustomers):
        for j in range(nCustomers):
            if i != j:
                rand = np.random.rand(1)
                
                if rand <= connectivitylevel:
                    ijcon[i][j] = 1
                    ijcon[j][i] = 1
                    
    # initial customer conditions
    cstateT = cState0*np.ones(nCustomers)
    cstateTpDT = cState0*np.ones(nCustomers)
    
    #initial storage unit system conditions
    sstateT = sState0
    sstateTpDT = sState0
     
    # function for overall sim will go here, lets define supporting 
    # functions first
    
    # the processes 
    timelimit = 100
    time = 0
    deltaT = 1
    lasttime = 0
    movieframes = 100
    totaldemand = 0
    totallosses = 0
    totalcost = 0  # initialize totalcost as 0, it will grow with each timestep
    totalsupplylosses = 0
    
    # Create positions of devices and supplier
    # position of EMS will always be origin, for now (randomly set to z = -5, need to ask Zohdi)
    sup_pos = np.array([0,-5,0],'float')
    
    if not locations:  # check that locations is an empty list
        
        # make positions an array of zeros to be filled in size: Nx3
        positions = np.array([[0] * 3] * N,'float')
        for i in range(N):    # loop through 
            positions[i][0] = bounds[0][0] + (bounds[0][1] - bounds[0][0])*np.random.rand()    # x min + x range * 0<=rand<=1
            positions[i][2] = bounds[1][0] + (bounds[1][1] - bounds[1][0])*np.random.rand()    # y min + y range * 0<=rand<=1
            positions[i][1] = 0
            
    else: # if locations has an input, then user input will decide positions of data centers
        positions = locations
    
    while time <= timelimit:
        # pre-simulation energies for each device
        initialE = np.random.rand(1,N)
        # generate target energy each device 
        targetE = np.random.rand(1,N)
        # generate uniform target energy in each device
        # use targ

        # generate energy consumed by each device
        Econ = np.random.rand(1,N)
        # generate energy generated by each device
        Egen = np.random.rand(1,N)

        # flux between devices -- THIS IS WHAT WE MANIPULATE IN THE GA
        interflux = interFlux(nCustomers,cIcJinterFluxBase)
        # customer state calculation
        cgen,cconsume = customerState(nCustomers,cgenbase,cConsumedbase)
        losses1 = 0
        losses2 = 0
        
        Pi,Pi_i,Min,pAve,totAve,sstateT = emsGA(G,S,P,C,minTol,N,w1,w2,w3,deltaT,initialE,targetE,Econ,Egen,
                                                positions,ijcon,targ,cIcJinterFluxBase)
        
        
        totalcost = totalcost + Pi[0]
        time += deltaT
    return totalcost, Pi_i
        

In [37]:
tc, pi_i = emsSIMwGA(20, [], [[-10, 10],[-10, 10]], .3, .4, .3)


In [38]:
tc

array([0.06056807])

In [41]:
totcost = emsSIM(20, [], [[-10, 10],[-10, 10]], .3, .4, .3)

In [42]:
totcost

4.73950743787966

In [17]:
r = np.random.rand(10,1)
ind = (np.argsort(r,axis=0)).astype(int)
rnew = copy.copy(r)
for i in range(len(ind)):
    r[i] = rnew[ind[i][0]]
print(rnew)
print(ind)
print(r)

[[0.73425038]
 [0.74872403]
 [0.07393278]
 [0.7324023 ]
 [0.98573987]
 [0.08199434]
 [0.55201886]
 [0.48224167]
 [0.58285584]
 [0.6951232 ]]
[[2]
 [5]
 [7]
 [6]
 [8]
 [9]
 [3]
 [0]
 [1]
 [4]]
[[0.07393278]
 [0.08199434]
 [0.48224167]
 [0.55201886]
 [0.58285584]
 [0.6951232 ]
 [0.7324023 ]
 [0.73425038]
 [0.74872403]
 [0.98573987]]


array([[0.03505459, 0.25268775, 0.45942019, 0.50232486, 0.70298163],
       [0.27198794, 0.40686736, 0.65330266, 0.68296545, 0.89975208],
       [0.12464354, 0.12864977, 0.17808056, 0.25811456, 0.63212233],
       [0.23970936, 0.53128901, 0.59366329, 0.73391841, 0.7364831 ]])

In [19]:
a[np.arange(0,3,2)+1,:,:]

array([[[0.78048998, 0.51722769, 0.85802992, 0.22598374, 0.16790306],
        [0.49584141, 0.91609668, 0.59782844, 0.48494513, 0.83426929],
        [0.57505142, 0.52915383, 0.82369005, 0.98616045, 0.16607987],
        [0.69534261, 0.94656066, 0.06112708, 0.23246201, 0.96372496]],

       [[0.39856489, 0.70178903, 0.67974775, 0.19875605, 0.7893135 ],
        [0.7456935 , 0.81428404, 0.28148447, 0.49814664, 0.9000104 ],
        [0.95638057, 0.75534682, 0.95150449, 0.35931475, 0.45582593],
        [0.84658237, 0.00751551, 0.39169419, 0.56923952, 0.32701628]]])

In [20]:
np.arange(0,3,2)

array([0, 2])

In [21]:
r = np.random.rand(10,1)
print(r)
r.sort()
print(r)

[[0.46092116]
 [0.86969399]
 [0.01125619]
 [0.86430372]
 [0.11428439]
 [0.89007966]
 [0.52658493]
 [0.23262589]
 [0.93443518]
 [0.39992756]]
[[0.46092116]
 [0.86969399]
 [0.01125619]
 [0.86430372]
 [0.11428439]
 [0.89007966]
 [0.52658493]
 [0.23262589]
 [0.93443518]
 [0.39992756]]


In [22]:

c = [3,0,2]
ind = (np.argsort(c)).astype(int)
for kk in range(len(ind)):
    b[kk] = b[ind[kk]]
b

NameError: name 'b' is not defined

In [ ]:
c = np.arange(1,5,2)
c+1

array([2, 4])

In [ ]:
a = np.random.rand(3,4)

In [ ]:
ndo = np.zeros([5,6-2-2])

In [ ]:
ndo

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])